In this notebook, we are going to implement a U-Net architecture for liver segmentation. We will tackle this problem in two main parts:

1. 2D U-Net segmentation
    1.1 Dataset
    1.2 Model
    1.3 Training
    1.4 Testing
2. 3D U-Net segmentation


In [1]:
from pathlib import Path
from Config import unzip, convert_folder
from LiverCTDataset import LiverCTDataset

import torch
import torch.nn as nn # neural networks functions
import torch.optim as optim # optimizer
from torch.utils.data import Dataset, DataLoader 
import torchvision
import torchvision.transforms as transforms 
from torchvision.datasets import ImageFolder
import timm # specific for image classification tasks

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

print(f'System version: {sys.version}')
print(f'PyTorch version: {torch.__version__}')
print(f'Torchvision version: {torchvision.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')

base_path = Path("C:/Users/HP/Desktop/PIMA/3Dircadb1")

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
PyTorch version: 2.6.0+cpu
Torchvision version: 0.21.0+cpu
NumPy version: 1.26.4
Pandas version: 2.2.3


For a better comprehension and a better visualization of the data, we unzip the PATIENT_DICOM, MASKS_DICOM and LABELLED_DICOM folders and convert the DICOM files into png or jpeg. In the convert_folder function and dicom_to_png_jpeg, specify which format you prefer.

There are 20 patients in the IRCAD dataset so we run the below code for each patient

In [2]:
for i in range(1, 21):
        current = f"3Dircadb1.{i}"
        patient = "PATIENT_DICOM"
        masks = "MASKS_DICOM"
        labelled = "LABELLED_DICOM"

        unzip(base_path / current / (masks + ".zip"), base_path / current / masks)
        convert_folder(base_path / current / masks / masks / "liver", base_path / current / masks / "converted")

        unzip(base_path / current / (patient + ".zip"), base_path / current / patient)
        convert_folder(base_path / current / patient / patient, base_path / current / patient / "converted")

        unzip(base_path / current / (labelled + ".zip"), base_path / current / labelled)
        convert_folder(base_path / current / labelled / labelled, base_path / current / labelled / "converted")

C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\MASKS_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\MASKS_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\PATIENT_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\PATIENT_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\LABELLED_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.1\LABELLED_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\MASKS_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\MASKS_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\PATIENT_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\PATIENT_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\LABELLED_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.2\LABELLED_DICOM
C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.3\MASKS_DICOM.zip is extracted to: C:\Users\HP\Desktop\PIMA\3Dircadb1\3Dircadb1.3\MASKS_DICOM
C:\User


| #  | 1st liver appearance | Last liver appearance | Total number of slices in between | Total patient slices |
|----|----------------------|-----------------------|-----------------------------------|----------------------|
| 1  | image_27             | image_124             | 98                                | 129                  |
| 2  | image_44             | image_157             | 114                               | 172                  |
| 3  | image_61             | image_182             | 122                               | 200                  |
| 4  | image_19             | image_85              | 67                                | 91                   |
| 5  | image_19             | image_137             | 119                               | 139                  |
| 6  | image_37             | image_132             | 96                                | 135                  |
| 7  | image_48             | image_145             | 98                                | 151                  |
| 8  | image_6              | image_121             | 116                               | 124                  |
| 9  | image_11             | image_99              | 89                                | 111                  |
| 10 | image_22             | image_120             | 99                                | 122                  |
| 11 | image_33             | image_128             | 96                                | 132                  |
| 12 | image_9              | image_247             | 239                               | 260                  |
| 13 | image_26             | image_115             | 90                                | 122                  |
| 14 | image_4              | image_107             | 104                               | 113                  |
| 15 | image_6              | image_120             | 115                               | 125                  |
| 16 | image_39             | image_151             | 113                               | 155                  |
| 17 | image_1              | image_113             | 113                               | 119                  |
| 18 | image_12             | image_70              | 59                                | 74                   |
| 19 | image_28             | image_68              | 41                                | 124                  |
| 20 | image_126            | image_210             | 85                                | 225                  |


2,12,18,20 were all black before normalization.

Creating and training a U-Net model with PyTorch for 2D & 3D semantic segmentation: Dataset building [1/4]

In [2]:
# exemple : lire les images du premier patient : image_27 et image_70
input = base_path / "3Dircadb1.1" / "PATIENT_DICOM" / "PATIENT_DICOM"
target =  base_path / "3Dircadb1.1" / "MASKS_DICOM" / "MASKS_DICOM" / "liver"
inputs = [input  / "image_46", input  / "image_70"]
targets = [target / "image_46", target / "image_70"]

training_dataset = LiverCTDataset(inputs=inputs, 
                           masks=targets,
                           transform=None)

training_dataloader = DataLoader(dataset=training_dataset, 
                                      batch_size=2, 
                                      shuffle=True)

x, y = next(iter(training_dataloader))
    
print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')

x = shape: torch.Size([2, 1, 512, 512]); type: torch.float32
x = min: -1024.0; max: 1023.0
y = shape: torch.Size([2, 512, 512]); class: tensor([  0, 255]); type: torch.int64
